# RECOMENDADOR DE PELICULAS

## DATASET: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system?select=ratings.csv

## PRIMERO SE  CREARA UNO ESPECIFICO

## POSTERIORMENTE SE CREARA UN ALGORITMO DONDE SE CREAN PARA TODOS LOS USUARIOS, SEGUN LOS 2 GENEROS QUE MAS LE GUSTAN Y MAS RECIENTES LAS PELICULAS.
#### ESTE ALGORITMO SE CREARA DE FORMA INDEPENDIENTE, VER ARCHIVO ALGORITMO_RECOMENDADOR

## EN ESTE ARCHIVO HACEMOS DE FORMA AUTOMATICA LA RECOMENDACION PARA TODOS LOS USUARIOS



## CARGA DE LAS LIBRERIAS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

from functools import total_ordering

import math

import matplotlib.pyplot as plt
puntaje = 3 ## PUNTUAJE MINIMO PARA CONSIDERAR QUE LE GUSTA AL USUARIO

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. CARGA DE LOS DATOS  DE PELICULAS Y HACEMOS UN ENCODER DE LOS GENEROS Y AÑADIMOS EL AÑO Y UN CONTADOR

In [ ]:
## CARGAMOS LAS PELICULAS
data = pd.read_csv('/content/drive/MyDrive/GRADO E IA Y BIGDATA/Colab Notebooks/4-GITHUB/7-RECOMENDADOR PELICULAS/movies.csv')

## SACAMOS UNA ESPECIE DE ONESHOOT ENCODER CON CADA GENERO POR PELICULA
for i in range(len(data)):
    particion = data.loc[i,"genres"].split(sep="|")
    for j in range(len(particion)):
        variable = particion[j]
        # print(variable)
        if variable not in data.columns:
            data[variable]=0
            data.loc[i,variable]=1
        else:
            data.loc[i,variable]=1

## SACAMOS DE MANERA INDEPENDIENTE EL AÑO
data['year'] = data['title'].str.extract(r'\((\d{4})\)$')


## ELIMINAMOS LOS NA DE AÑO
data = data.dropna()
data['year']= data['year'].astype(int)

## AÑADIMOS UNA COLUMNA CON 1 PARA CONTAR DE UNA FORMA MAS RAPIDA
data['contador'] = 1
print("#" * 60)
print("##HEAD DATA##")
print(data.head())

## CARGAMOS LOS USUARIOS
ratings = pd.read_csv('/content/drive/MyDrive/GRADO E IA Y BIGDATA/Colab Notebooks/4-GITHUB/7-RECOMENDADOR PELICULAS/ratings.csv')
print("#" * 60)
print("##HEAD RATING##")
print(ratings.head())
print(ratings.tail())
## CREAMOS UN DICCIONARIO CON LOS GENEROS
generodict= {0:'Adventure', 1:'Animation', 2:'Children', 3:'Comedy', 4:'Fantasy',
             5:'Romance', 6:'Drama', 7:'Action', 8:'Crime', 9:'Thriller',
             10:'Horror', 11:'Mystery', 12:'Sci-Fi', 13:'IMAX', 14:'Documentary',
             15:'War', 16:'Musical', 17:'Western', 18:'Film-Noir'}

## SACAMOS LOS GUSTOS POR USUARIO
# for user in ratings['userId'].unique():  ## EMPLEARIAMOS ESTO EN PRODUCCION, PARA SOLO EJEMPLO USAREMOS LA LINEA DE ABAJO
## O APLICAMOS UN INPUT E INTRODUCIMOS EL INPUT
for user in range(1,11):
    userdata= ratings[ratings['userId']==user]
    usergustos = data[data['movieId'].isin(userdata['movieId'])].copy()
    usergustos['puntos'] = ratings[ratings['userId']==user]['rating']

## PUNTUAMOS LOS GUSTOS
    genero=[]
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Adventure'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Animation'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Children'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Comedy'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Fantasy'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Romance'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Drama'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Action'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Crime'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Thriller'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Horror'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Mystery'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Sci-Fi'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['IMAX'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Documentary'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['War'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Musical'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Western'].sum())
    genero.append(usergustos[usergustos['puntos']>=puntaje]['Film-Noir'].sum())
    total = (usergustos[usergustos['puntos']>=puntaje]['contador'].sum())

## SELECCIONAMOS LOS GUSTOS QUE ESTAN EN EL 20% DE LAS PELICULAS
    abuscar=[]
    for i in range(len(genero)):
        if i < 19:
            if genero[i] > (total * 0.20) :
                abuscar.append(1)
            else:
                abuscar.append(0)

## CONTAMOS CUANTOS GUSTOS SUPERIORES AL 20%
    totalabuscar = 0
    for i in range(len(abuscar)):
        if abuscar[i] == 1:
            totalabuscar = totalabuscar + 1

    ## SI SON MAS DE 2 SELECCIONAMOS 2 LOS MAS IMPORTANTES
    if totalabuscar > 2:
        # Crear lista de pares (valor, índice_original)
        indexed_genero = [(genero[i], i) for i in range(len(genero))]
        # Ordenar por valor descendente
        indexed_genero.sort(reverse=True)
        # Obtener los índices originales de los 3 primeros
        top_indices = [indexed_genero[i][1] for i in range(2)]

        # Actualizar abuscar
        for i in range(len(abuscar)):
            abuscar[i] = 1 if i in top_indices else 0

    ## BUSCAMOS LOS 2 GUSTOS MASIMPORTANTES EN TODAS LAS PELICULAS
    peliculas= data[(data['Adventure'] == abuscar[0]) \
                    & (data['Animation'] == abuscar[1])  \
                    & (data['Children'] == abuscar[2]) \
                    & (data['Comedy'] == abuscar[3]) \
                    & (data['Fantasy'] == abuscar[4]) \
                    & (data['Romance'] == abuscar[5]) \
                    & (data['Drama'] == abuscar[6]) \
                    & (data['Action'] == abuscar[7]) \
                    & (data['Crime'] == abuscar[8]) \
                    & (data['Thriller'] == abuscar[9]) \
                    & (data['Horror'] == abuscar[10]) \
                    & (data['Mystery'] == abuscar[11]) \
                    & (data['Sci-Fi'] == abuscar[12]) \
                    & (data['IMAX'] == abuscar[13]) \
                    & (data['Documentary'] == abuscar[14]) \
                    & (data['War'] == abuscar[15]) \
                    & (data['Musical'] == abuscar[16]) \
                    & (data['Western'] == abuscar[17]) \
                    & (data['Film-Noir'] == abuscar[18])

            ]

    ## SELECCIONAMOS DE PELICULAS LAS DEL AÑO MAS CERCANO AL ACTUAL
    cartelera = peliculas[peliculas['year']==peliculas['year'].max()]
    if len(cartelera['title']) > 20:
        cartelera1 = True
    else:
        cartelera1= False
    ## IMPRIMIMOS LAS PELICULAS DEL USUARIO
    print("#" * 60)
    print(f"##USER {user}##")
    if cartelera1 == True:
        print("DEMASIADAS PELICULAS O SIN DATOS")
    else:
        print(cartelera['title'])

############################################################
##HEAD DATA##
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  Adventure  Animation  \
0  Adventure|Animation|Children|Comedy|Fantasy          1          1   
1                   Adventure|Children|Fantasy          1          0   
2                               Comedy|Romance          0          0   
3                         Comedy|Drama|Romance          0          0   
4                                       Comedy          0          0   

   Children  Comedy  Fantasy  Romance  Drama  ...  Sci-Fi  IMAX  Documentary  \
0         1       1        1        0      0  ...       0     0            0   
1         1       0        1   